In [1]:
import time
import os
import shutil
import sys
import h5py
import re
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
import random                                 as rnd
from matplotlib                 import pyplot as plt 

print("="*50)
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [2]:
def write_predictiondata(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):
    
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]
                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [3]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

ExcitType           = 'KInel'
TTranVec            = [20000.0]
MinValueTrain       = 1.e-20

Molecules           = ['N2','N2']
Atoms               = ['N','N']
System = 'N3_NASA'

QCTKineticFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D_semi/database/kinetics/'+System+'/'


PathToHDF5File  =  WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
Molecules       = ['N2','N2'] 
PathToDiatFile  = [WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
                   WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   


In [4]:
sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

NMolecules = len(Molecules)
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):
    DiatDataTemp = read_diatdata(PathToDiatFile[iMol], Molecules[iMol], np.array(TTranVec), np.array(TTranVec))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = DiatDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp


In [5]:
#========================================================================================================
def read_kexcitdata_PLATO(PathToKineticFile, TTra):
    print(PathToKineticFile)
    Rates = 1e-16*np.ones((NLevels[0],NLevels[0]))
    i2j_dict = {}
    with open(PathToKineticFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('N2|N|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            jIdx    = int(linestemp[8]);
            RateTemp = float(linestemp[12]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates, i2j_dict

In [6]:
if(ExcitType == 'KInel'): 
    PathToKineticFile  = QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Inel.dat'
elif(ExcitType == 'KExch'): 
    PathToKineticFile  = QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Exch_Type1.dat'
    
QCTRates, i2j_dict = read_kexcitdata_PLATO(PathToKineticFile,TTranVec[0])

/home/venturi/WORKSPACE//Air_Database/Run_0D_semi/database/kinetics/N3_NASA//T20000K/Inel.dat


In [7]:
QCTKineticFldr1 = WORKSPACE_PATH + '/Air_Database/Run_0D_diffAccuracy/database_1em16/'
try:
    os.makedirs(QCTKineticFldr1)
except OSError as e:
    pass

QCTKineticFldr1 = WORKSPACE_PATH + '/Air_Database/Run_0D_diffAccuracy/database_1em16/kinetics/'
try:
    os.makedirs(QCTKineticFldr1)
except OSError as e:
    pass

QCTKineticFldr1 = WORKSPACE_PATH + '/Air_Database/Run_0D_diffAccuracy/database_1em16/kinetics/'+System+'/'
try:
    os.makedirs(QCTKineticFldr1)
except OSError as e:
    pass

for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = QCTKineticFldr1 + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        jIdxVec = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
        
        if(ExcitType == 'KInel'):
            KInel_NN_FWD = QCTRates[iIdx,jIdxVec]
            csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
        elif(ExcitType == 'KExch'):   
            KExch_NN_FWD = QCTRates[iIdx,jIdxVec]
            csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)
                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 2/9399 [00:00<08:22, 18.69it/s]

[SurQCT]:    TTran =  20000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_diffAccuracy/database_1em16/kinetics/N3_NASA//T20000K/Inel.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 9399/9399 [09:40<00:00, 16.19it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_diffAccuracy/database_1em16/kinetics/N3_NASA//T20000K/Inel.dat


array([[1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15],
       [1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15],
       [1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15],
       [1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15],
       [1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15]])